# Cross validation

## Imports

In [18]:
import pandas as pd
from sqlalchemy import create_engine

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()


Exception during reset or similar
Traceback (most recent call last):
  File "c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sqlalchemy\pool\base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sqlalchemy\pool\base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sqlalchemy\engine\default.py", line 667, in do_rollback
    dbapi_connection.rollback()
  File "src\pymssql\_pymssql.pyx", line 316, in pymssql._pymssql.Connection.rollback
  File "src\pymssql\_pymssql.pyx", line 300, in pymssql._pymssql.Connection.rollback
  File "src\pymssql\_mssql.pyx", line 1056, in pymssql._mssql.MSSQLConnection.execute_non_query
  File "src\pymssql\_mssql.pyx", line 1077, in pymssql._mssql.MSSQLConnection.execute_non_query
  File "src\pymssql\_mssql.pyx", line 1251, in pymssql._mssql.MSSQLConnection.format_and_run_query
  File "s

## EXAMPLE: TRAIN FROM 2021-12, PREDICT FOR 2022-01

In [19]:
df = pd.read_sql(
    """
    select ticker, date, agr, bm, idiovol, mom12m, roeq, ret
    from data
    where date in ('2021-12', '2022-01')
    """, 
    conn
)
features = ["agr", "bm", "idiovol", "mom12m", "roeq"]

## TRANSFORM EACH CROSS-SECTION

In [6]:
from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer(output_distribution="normal")

def qtxs(d):
    x = qt.fit_transform(d)
    return pd.DataFrame(x, columns=d.columns, index=d.index)

df[features + ["ret"]] = df.groupby(
  "date", 
  group_keys=False
)[features + ["ret"]].apply(qtxs)

## FITTING A RANDOM FOREST

In [7]:
from scipy.stats import spearmanr
from sklearn.metrics import make_scorer

scorer = make_scorer(
    lambda a, b: spearmanr(a, b).statistic, 
    greater_is_better=True
)

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

df = df.dropna()

X = df[df.date=='2021-12'][features]
y = df[df.date=='2021-12']["ret"]

model = RandomForestRegressor()

cv = GridSearchCV(
  model,
  param_grid={
    "max_depth": range(1, 11)
  },
  scoring=scorer
)
_ = cv.fit(X, y)

c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\Risto Trajanov\AppData\Local\Temp\ipykernel_123248\1467376756.py", line 5, in <lambda>
    lambda a, b: spearmanr(a, b).statistic,
AttributeError: 'SpearmanrResult' object has no attribute 'stati

In [17]:
scores = [x for x in cv.cv_results_['mean_test_score']]
print(scores)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


## GRIDSEARCHCV FOR MULTI-LAYER PERCEPTRON

In [ ]:
param_grid={
    "hidden_layer_sizes": [
        [16, 8, 4, 2], 
        [16, 8, 4],
        [8, 4, 2], 
        [16, 8],
        [16, 4],
        [8, 4],
        [4, 4],
        [4, 2]
    ]
}

In [ ]:
cv = GridSearchCV(
  MLPRegressor(max_iter=500)
  param_grid=param_grid,
  scoring=scorer
)
_ = cv.fit(X, y)